[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PyGIS222/Fall2019/blob/master/LessonM62_Shapely.ipynb)

(Note for Google Colab: Not all packages used in this notebook are available by default on Google Colab. Google Colab provides an install option, however, this has to be repeated each time you start a new session on their servers.)

## Notebook Lesson 6.2

# Module Shapely

This Jupyter Notebook is part of Module 6 of the course GIS222 (Fall2019). This lesson is preparing you for the final project of the course. Carefully study the content of this Notebook and use the chance to reflect the material through the interactive examples.

### Introduction

The purpose of this notebook is to introduce some content of the Python module **Shapely**, which is necessary for the final project.

### Sources
Documentation pages of the module [Shapely](https://shapely.readthedocs.io/en/stable/index.html) and the [Python-GIS page about Shapely](https://automating-gis-processes.github.io/CSC18/lessons/L4/point-in-polygon.html).

---


# Selecting Points Inside a Boundary: Shapely

In previous lesson parts, we have been loading imagery like a shaded relief, plot them and sliced them. Later, we will be doing something like this with science data and in those cases also calculate some map statistics. However, so far we have always been using a certain bounding box for our maps. In a more realistic scenario, we would be interested to retrieve data content for a certain geographical or political area, like an ocean region (like the Atlatnic), a river basin, a country or a state.

Such boundaries are not lat/lon blocks, but defined by more complex polygons and we cannot simply slice them out of the arrays. To be able to select unregularly shaped region from a data array, we will peak into a module, which provides us with tools to perform exactly that: selecting raster data within a region. As you will see later, this is a very typical GIS problem, which reduces to the problem of finding out whether a **point falls inside a polygon**. But while this is an easy gasp if we can visually look at the data in a map, here we would like to code this in Python!

Below a brief insight into the module shapely. This will also be an outlook for GIS322, where this module will be discussed in more detail, as it provides basic procedures for a lot of GIS applications.

<img src="./img/M62_Shapely.png" width="400" />

Figure 1: *Fundamental geometric objects that can be used in Python with Shapely module*


## Creating Points and Lines using Shapely

The most fundamental geometric objects are Points, Lines and Polygons which are the basic ingredients when working with spatial data in vector format. Python has a specific module called Shapely, which can be used to create and work with planar geometric objects. There are many useful functionalities that you can do with Shapely such as:

* Create a Line or Polygon from a Collection of Point geometries
* Calculate areas/length/bounds etc. of input geometries
* Make geometric operations based on the input geometries such as Union, Difference, Distance etc.
* Make spatial queries between geometries such Intersects, Touches, Crosses, Within etc.

Geometric Objects consist of coordinate tuples where:

* Point-objects represent a single point in space. Points can be either two-dimensional (x, y) or three dimensional (x, y, z).
* LineString-objects (i.e. a line) represent a sequence of points joined together to form a line. Hence, a line consist of a list of at least two coordinate tuples
* Polygon objects represents a filled area that consists of a list of at least three coordinate tuples that forms the outerior ring and a (possible) list of hole polygons.

Here we will look only into selected parts of the module: **Points and Polygons** and related functions `within()` and `contain()`, which are necessary to perform the final project.

### Point

During module 5, we were writing our own Point class. Something similar, a Point class, is provided by the shapely package but with many more methods and attributes.

Creating a shapely Point is easy, you pass x and y coordinates into a Point()-object:

In [27]:
# Import geometric object Point from shapely module
from shapely.geometry import Point, Polygon

In [28]:
# Create Point geometric object(s) with coordinates
point1 = Point(2.2, 4.2)
point2 = Point(7.2, -25.1)
point3 = Point(9.26, -2.456)

Let’s see what the variables look like

In [29]:
# printing point 1 to screen
print(point1)

POINT (2.2 4.2)


In [30]:
# What is the type of the point?
print(type(point1))

<class 'shapely.geometry.point.Point'>


We can see that the type of the point is shapely’s Point which is represented in a specific format that is based on GEOS C++ library that is one of the standard libraries in GIS.

### Point attributes and functions

The point-object has some built-in attributes that can be accessed and also some useful functionalities. One of the most useful ones are the ability to extract the coordinates of a Point and calculate the Euclidian distance between points.

Extracting the coordinates of a Point can be done in a couple of different ways:

In [31]:
# Get the coordinates
point_coords = point1.coords 

# What is the type of this?
type(point_coords) 

shapely.coords.CoordinateSequence

Ok, we can see that the output is a Shapely CoordinateSequence. Let’s see how we can get out the actual coordinates:

In [32]:
# Get x and y coordinates
xy = point_coords.xy

# Get only x coordinates of Point1
x = point1.x

# Whatabout y coordinate?
y = point1.y

What is inside?

In [33]:
# printing received coordinates of the point
print(xy)

(array('d', [2.2]), array('d', [4.2]))


In [34]:
# printing received x coordinate of the point
print(x)

2.2


In [35]:
# printing received y coordinate of the point
print(y)

4.2


So we can see that the our xy variable contains a tuple where x and y are stored inside of numpy arrays. However, our x and y variables are plain decimal numbers.

It is also possible to calculate the distance between points which can be useful in many applications. The returned distance is based on the projection of the points (degrees in WGS84, meters in UTM).

In [36]:
# Calculate the distance between point1 and point2
point_dist = point1.distance(point2)
print("Distance between the points is {0:.2f} decimal degrees".format(point_dist))

Distance between the points is 29.72 decimal degrees


## Polygon

Creating a shapely-object Polygon continues the same logic of how Point was created but Polygon objects only accept coordinate-tuples as input. A shapely Polygon needs at least three coordinate-tuples:

In [37]:
# Create a Polygon from the coordinates
poly = Polygon([(2.2, 4.2), (7.2, -25.1), (9.26, -2.456)])

# We can also use our previously created Point objects (same outcome)
# --> notice that Polygon object requires x,y coordinates as input
poly2 = Polygon([[p.x, p.y] for p in [point1, point2, point3]])

# Geometry type can be accessed as a String
poly_type = poly.geom_type

# Using the Python's type function gives the type in a different format
poly_type2 = type(poly)

Let's see how our Polygon looks like:

In [38]:
# printing out both polygons
print(poly)
print(poly2)

# printing the polygon type as text and how Python shows is
print("Geometry type as text:", poly_type)
print("Geometry how Python shows it:", poly_type2)

POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
POLYGON ((2.2 4.2, 7.2 -25.1, 9.26 -2.456, 2.2 4.2))
Geometry type as text: Polygon
Geometry how Python shows it: <class 'shapely.geometry.polygon.Polygon'>


### Polygon attributes and functions

We can again access different attributes that are really useful such as area, centroid, bounding box, exterior, and exterior-length of the Polygon. Here some examples:

In [39]:
# Define a polygon that circumferes a mercator projected global map
world = Polygon([(-180, 90), (-180, -90), (180, -90), (180, 90)])

# Get the centroid of the Polygon
world_centroid = world.centroid

# Get the area of the Polygon
world_area = world.area

# Get the bounds of the Polygon (i.e. bounding box)
world_bbox = world.bounds

# Get the exterior of the Polygon
world_ext = world

# Get the length of the exterior
world_ext_length = world_ext.length

In [40]:
print("Poly centroid: ", world_centroid)
print("Poly Area: ", world_area)
print("Poly Bounding Box: ", world_bbox)
print("Poly Exterior: ", world_ext)
print("Poly Exterior Length: ", world_ext_length)

Poly centroid:  POINT (-0 -0)
Poly Area:  64800.0
Poly Bounding Box:  (-180.0, -90.0, 180.0, 90.0)
Poly Exterior:  POLYGON ((-180 90, -180 -90, 180 -90, 180 90, -180 90))
Poly Exterior Length:  1080.0


Shapely also provides the option of creating lines. But we will stop at this point and postpone further content on shapely objects to GIS322. 

## Point in Polygon & Intersect

Finding out if a certain **point is located inside or outside of an area is a fundamental geospatial operation** that is often used, for example, **to select data based on location**. Such spatial queries are one of the typical first steps of the workflow when doing spatial analysis. 

Computationally, detecting if a point is inside a polygon is most commonly done using a specific formula called Ray Casting algorithm. Luckily, we do not need to create such a function ourselves for conducting the **Point in Polygon (PIP)** query. Instead, we can take advantage of Shapely’s binary predicates that can evaluate the topolocical relationships between geographical objects, such as the PIP as we’re interested here.

There are basically two ways of conducting PIP in Shapely:

* using a function called .within() that checks if a point is within a polygon
* using a function called .contains() that checks if a polygon contains a point

Let’s first again create a Polygon using a list of coordinate-tuples and a couple of Point objects:

In [41]:
# Create Point objects
p1 = Point(24.952242, 60.1696017)
p2 = Point(24.976567, 60.1612500)

# Create a Polygon
coords = [(24.950899, 60.169158), (24.953492, 60.169158), (24.953510, 60.170104), (24.950958, 60.169990)]
poly = Polygon(coords)

Let's check what we have:

In [42]:
print(p1)

POINT (24.952242 60.1696017)


In [43]:
print(p2)

POINT (24.976567 60.16125)


In [44]:
print(poly)

POLYGON ((24.950899 60.169158, 24.953492 60.169158, 24.95351 60.170104, 24.950958 60.16999, 24.950899 60.169158))


Now, let’s check if those points are `within()` the polygon

In [45]:
# Check if p1 is within the polygon using the within function
p1.within(poly)

True

In [46]:
# Check if p2 is within the polygon
p2.within(poly)

False

We can see that the first point seems to be inside that polygon and the other one doesn’t.

In fact, the first point is close to the center of the polygon as we can see:

In [47]:
# Our point
print(p1)

# The centroid
print(poly.centroid)

POINT (24.952242 60.1696017)
POINT (24.95224242849236 60.16960179038188)


It is also possible to do PIP other way around, i.e. to check if polygon contains a point:

In [48]:
# Does polygon contain p1?
poly.contains(p1)

True

In [49]:
# Does polygon contain p2?
poly.contains(p2)

False

Both ways of checking the spatial relationship results in the same outcome. But which one should you use then? Well, it depends:

If you have **many points and just one polygon** and you try to find out which one of them is inside the polygon:
    
* You need to iterate over the points and check, one at a time, if it is `within()` the polygon specified

If you have **many polygons and just one point** and you want to find out which polygon contains the point

* You iterate over the polygons until you find a polygon that `contains()` the point specified (assuming there are no overlapping polygons).

### Conclusion

The module shapely provides us with data object types for points and polygons (and more). It also comes with function to perform a Point in Polygon (PIP) analysis.
This is definitely not all to know about the module shapely, but the most important functions to move on to analyze our sea surface temperatures for the atlantic ocean only!